# Most frequent words

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
import numpy as np
from pattern.text.es import singularize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from share import select_secondary_keywords_ngrams
from tqdm.notebook import tqdm
import pickle
tqdm.pandas()

In [2]:
all_df = pd.read_csv('all_df.csv')
all_df.shape

(352834, 8)

In [8]:
all_df_2023 = all_df.loc[all_df['Anio']==2023]
all_df_2023.to_csv('/Users/almacuevas/work_projects/conferencias_matutinas_amlo/all_df_2023.csv')

In [21]:
def split_content(content):
    if not content:
        return []
    return ' '.join(re.split(r"\s*[:?!;|,-.]\s*", content))

In [22]:
all_df["preprocess"] = all_df.progress_apply(lambda row: split_content(row.Texto), axis=1)
all_df

  0%|          | 0/352834 [00:00<?, ?it/s]

,Unnamed: 0,Participante,Texto,Sentimiento,Palabras,Dia,Mes,Anio,preprocess
0,0,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Informarles que el día de hoy voy a enviar al...,0.001959,58,6,12,2018,Informarles que el día de hoy voy a enviar al...
1,1,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Les doy a conocer los nombres de quienes están...,0.771382,10,6,12,2018,Les doy a conocer los nombres de quienes están...
2,2,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,La abogada Loretta Ortiz. Ella es egresada de ...,0.034682,27,6,12,2018,La abogada Loretta Ortiz Ella es egresada de l...
3,3,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"También, la licenciada Celia Maya García. Ella...",0.003624,46,6,12,2018,También la licenciada Celia Maya García Ella e...
4,4,-PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Y el tercer abogado propuesto es el licenciado...,0.017006,39,6,12,2018,Y el tercer abogado propuesto es el licenciado...
...,...,...,...,...,...,...,...,...,...
352829,352829,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Y va a seguir ayudando Ricardo en Coahuila par...,0.000000,38,15,12,2022,Y va a seguir ayudando Ricardo en Coahuila par...
352830,352830,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Esto me lo plantearon en una reunión que tuve ...,0.000000,67,15,12,2022,Esto me lo plantearon en una reunión que tuve ...
352831,352831,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Entonces, él va a seguir ayudándonos en esto.",0.000000,8,15,12,2022,Entonces él va a seguir ayudándonos en esto
352832,352832,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Y con mucha responsabilidad, con mucha madurez...",0.000000,44,15,12,2022,Y con mucha responsabilidad con mucha madurez ...


In [23]:
all_df.to_csv('/Users/almacuevas/work_projects/conferencias_matutinas_amlo/all_df_preprocess.csv')

In [28]:
df_year = all_df.loc[all_df["Anio"]==2018][0:100]
frequent_words = select_secondary_keywords_ngrams(df_year.preprocess, 5, 5)  #[0:max_n]
frequent_words

,term,rank
0,sara pablo radio fórmula presidente,1.000000
1,eso puede seguir pasando país,1.000000
2,jorge monroy periódico el economista,1.000000
3,nosotros vamos respaldar compromisos inversion...,1.000000
4,en días creo va comenzar,1.000000
...,...,...
935,adulto mayor ciudad méxico muchas,0.188982
936,méxico muchas acciones eso general,0.188982
937,financiar programa bienestar contratación jóvenes,0.188982
938,estudiantes niveles escolaridad aumenta pensión,0.188982


# Unsupervised classification of all conversations

In [2]:
year_label = '2019'
frequent_words = pd.read_csv(f'/Users/almacuevas/work_projects/conferencias_matutinas_amlo/spacy_graph_metainfo/world_data_lemma_set_{year_label}.csv')
frequent_words

,Unnamed: 0.1,Unnamed: 0,term,rank,parsed_doc,comment_tokens,comment_lemma,pos_pos
0,0,0,gracias,1756.253788,gracias,gracias,gracias,NOUN
1,1,1,presidente,1361.448084,presidente,presidente,presidente,NOUN
2,2,6,días,909.247868,día,día,día,NOUN
3,3,7,buenos,843.438919,buen,buen,buen,ADJ
4,4,10,señor,699.668977,señor,señor,señor,NOUN
...,...,...,...,...,...,...,...,...
20092,20092,40622,nath,0.108223,nath,nath,nath,NOUN
20093,20093,40623,jinpin,0.108223,jinpin,jinpin,jinpin,ADJ
20094,20094,40624,theresa,0.108223,theresa,theresa,theresa,NOUN
20095,20095,40627,comercializadores,0.086605,comercializador,comercializador,comercializador,ADJ


In [5]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words("spanish"))
X = vectorizer.fit_transform(frequent_words["term"])
true_k = 2 # Number of groups
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :100]:
        print(' %s' % terms[ind])

Cluster 0:
 útiles
 dotado
 dormilona
 dormilón
 dormitorios
 dorándonos
 dosis
 dotación
 dotadas
 dotados
 dormida
 dotes
 down
 doña
 draconiana
 dragado
 dragados
 dragas
 dormido
 doria
 duplicaciones
 donado
 domésticas
 doméstico
 domésticos
 don
 donaciones
 donación
 donadas
 donají
 dorenlas
 donantes
 donatarias
 donativos
 donceles
 dorada
 dorado
 dorados
 dragón
 drama
 dramatizada
 dudosa
 dublín
 duc
 ducto
 ductos
 duda
 dudas
 dudley
 duelo
 dramática
 dueña
 dueñas
 dueño
 dueños
 dulce
 dulzón
 dumping
 dubinsky
 dual
 drástico
 drásticas
 dramáticas
 dramático
 dramáticos
 drenado
 drenaje
 drenajes
 droga
 drogada
 drogadas
 drogadicción
 drogados
 drogas
 dron
 drones
 drástica
 doméstica
 domo
 dominio
 docentes
 doblegados
 dobles
 doblete
 docena
 docenas
 docencia
 docente
 doctor
 documentado
 doctora
 doctorados
 doctores
 doctrina
 doctrinas
 documentación
 documentada
 doble
 dobladito
 diódoro
Cluster 1:
 cariñosos
 útiles
 dotadas
 dormilona
 dormilón
 